In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

csv_file_path = "IRIS.csv"  
data = pd.read_csv(csv_file_path)

features = data.iloc[:, :-1].values 
labels = data.iloc[:, -1].values  

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

scaler = StandardScaler()
features = scaler.fit_transform(features)

In [5]:
np.savez("iris_data.npz", features=features, labels=labels)

In [21]:
loaded_data = np.load("/kaggle/working/iris_data.npz")
features_loaded = loaded_data["features"]
labels_loaded = loaded_data["labels"]

print(features_loaded.shape, labels_loaded.shape)

(150, 4) (150,)


In [22]:
dataset = tf.data.Dataset.from_tensor_slices((features_loaded, labels_loaded))

In [23]:
num_data_valid = int(0.1 * len(features_loaded))
num_data_test = int(0.1 * len(features_loaded))

In [24]:
buff_size = 100
dataset = dataset.shuffle(buffer_size=buff_size)

In [25]:
valid_data = dataset.take(num_data_valid)
test_data = dataset.skip(num_data_valid).take(num_data_test)
train_data = dataset.skip(num_data_valid + num_data_test)

In [26]:
batch_size = 32
train_data = train_data.batch(batch_size)
valid_data = valid_data.batch(num_data_valid)
test_data = test_data.batch(num_data_test)

In [27]:
valid_inputs, valid_targets = next(iter(valid_data))

In [28]:
input_size = 4
output_size = 3
hidden_size = 128

model = tf.keras.Sequential([
    tf.keras.Input(shape = (input_size,)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hidden_size, activation= 'relu'),
    tf.keras.layers.Dense(hidden_size, activation= 'relu'),
    tf.keras.layers.Dense(hidden_size, activation= 'relu'),
    tf.keras.layers.Dense(output_size, activation= 'softmax'),
])

In [29]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.001 * 0.95 ** epoch)
model.compile(optimizer=optimizer, loss = 'sparse_categorical_crossentropy', metrics= ['accuracy'])

In [30]:
epochs = 20
model.fit(train_data, 
          epochs=epochs, 
          validation_data=(valid_inputs, valid_targets), 
          callbacks=[early_stopping, lr_scheduler], 
          verbose = 2)

Epoch 1/20
4/4 - 1s - 347ms/step - accuracy: 0.5583 - loss: 1.0203 - val_accuracy: 0.4000 - val_loss: 0.9999 - learning_rate: 0.0010
Epoch 2/20
4/4 - 0s - 9ms/step - accuracy: 0.7333 - loss: 0.8304 - val_accuracy: 0.6000 - val_loss: 0.8746 - learning_rate: 9.5000e-04
Epoch 3/20
4/4 - 0s - 9ms/step - accuracy: 0.8083 - loss: 0.6693 - val_accuracy: 0.6000 - val_loss: 0.7934 - learning_rate: 9.0250e-04
Epoch 4/20
4/4 - 0s - 9ms/step - accuracy: 0.8167 - loss: 0.5601 - val_accuracy: 0.6000 - val_loss: 0.7218 - learning_rate: 8.5737e-04
Epoch 5/20
4/4 - 0s - 9ms/step - accuracy: 0.8333 - loss: 0.4514 - val_accuracy: 0.6000 - val_loss: 0.6589 - learning_rate: 8.1451e-04
Epoch 6/20
4/4 - 0s - 9ms/step - accuracy: 0.8500 - loss: 0.3927 - val_accuracy: 0.6667 - val_loss: 0.6038 - learning_rate: 7.7378e-04
Epoch 7/20
4/4 - 0s - 9ms/step - accuracy: 0.8500 - loss: 0.3430 - val_accuracy: 0.7333 - val_loss: 0.5098 - learning_rate: 7.3509e-04
Epoch 8/20
4/4 - 0s - 10ms/step - accuracy: 0.8833 - loss

In [36]:
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9333 - loss: 0.2201
Test Accuracy: 93.33%
